Importing required libraries

In [ ]:
import cv2
import imutils
from imutils.object_detection import non_max_suppression
import numpy as np
import requests
import time
from urllib.request import urlopen

ThingSpeak Configuration
channel_id: ThingSpeak channel ID
WRITE_API: ThingSpeak write API key
BASE_URL: Base URL for sending data to ThingSpeak

In [ ]:
channel_id =  2273329
WRITE_API  = 'ZZE21YW6F9KHEI2H'
BASE_URL = "https://api.thingspeak.com/update?api_key={}".format(WRITE_API)

This code initializes a HOG descriptor and configures it to use a pre-trained SVM detector for pedestrian detection. This allows you to use the HOG descriptor to detect people in images by applying the SVM classifier to the extracted HOG features.

In [ ]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

The below function provides a pipeline for detecting pedestrians in an image using HOG and SVM, and it incorporates non-maximum suppression to improve the accuracy of the detections.

In [ ]:
def detector(image):
    image = imutils.resize(image, width=min(400, image.shape[1]))
    clone = image.copy()
    rects, weights = hog.detectMultiScale(image, winStride=(4, 4), padding=(8, 8), scale=1.05)
    for (x, y, w, h) in rects:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    result = non_max_suppression(rects, probs=None, overlapThresh=0.3)  # Lowered overlapThresh
    return result

The below record function provides a way to continuously monitor an environment using a webcam, detect people using HOG and SVM, and send the number of detected people to ThingSpeak at regular intervals.

In [ ]:
def record(sample_time=5):
    print("recording")
    camera = cv2.VideoCapture(0)
    init = time.time()

    if sample_time < 3:
        sample_time = 3

    while True:
        print("cap frames")
        ret, frame = camera.read()
        frame = imutils.resize(frame, width=min(400, frame.shape[1]))
        result = detector(frame.copy())
        result1 = len(result)
        print(result1)

        for (xA, yA, xB, yB) in result:
            cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)

        cv2.imshow("Frame", frame)
        cv2.waitKey(1)

        if time.time() - init >= sample_time:
            thingspeakHttp = BASE_URL + "&field1={}".format(result1)
            print(thingspeakHttp)
            conn = urlopen(thingspeakHttp)
            print("Sending result")
            init = time.time()

    camera.release()
    cv2.destroyAllWindows()

The main function acts as a bridge between the script's modules and the execution of the core functionality. It ensures that the record function is only called when the script is run as the main program, preventing unintended side effects if the script is imported as a module into another program.

In [ ]:
def main():
    record()

if __name__ == '__main__':
    main()